<a href="https://colab.research.google.com/github/thazin31086/WebTrace/blob/master/Skip_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
#https://towardsdatascience.com/learn-word2vec-by-implementing-it-in-tensorflow-45641adaf2ac
import numpy as np
import tensorflow as tf
corpus_raw = 'The cat sat on the mat'

# convert to lower case
corpus_raw = corpus_raw.lower()


words = []
for word in corpus_raw.split():   
        words.append(word)
words = set(words) # so that all duplicate words are removed

word2int = {}
int2word = {}
vocab_size = len(words) # gives the total number of unique words
print(vocab_size)
for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

    
 # raw sentences is a list of sentences.
raw_sentences = corpus_raw.split('.')
sentences = []
for sentence in raw_sentences:
    sentences.append(sentence.split())
    
print(sentences)


data = []
WINDOW_SIZE = 2
for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])
                
print(data)


# function to convert numbers to one hot vectors
def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp
  
x_train = [] # input word
y_train = [] # output word
for data_word in data:
    x_train.append(to_one_hot(word2int[ data_word[0] ], vocab_size))
    y_train.append(to_one_hot(word2int[ data_word[1] ], vocab_size))
# convert them to numpy arrays
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)


print(x_train.shape, y_train.shape)
# meaning 18 training points, where each point has 5 dimensions

# making placeholders for x_train and y_train
x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size))


EMBEDDING_DIM = vocab_size # you can choose your own number
W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
hidden_representation = tf.add(tf.matmul(x,W1), b1)

W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2))

#to train it:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) #make sure you do this!
# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), reduction_indices=[1]))
# define the training step:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)
n_iters = 10000
# train for n_iter iterations
for _ in range(n_iters):
    sess.run(train_step, feed_dict={x: x_train, y_label: y_train})
    #rint('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: x_train, y_label: y_train}))
    
vectors = sess.run(W1 + b1)
print(vectors[ word2int['cat'] ])

#quick function to find the closest vector to a given vector. Beware, it’s a dirty implementation.
def euclidean_dist(vec1, vec2):
    return np.sqrt(np.sum((vec1-vec2)**2))
def find_closest(word_index, vectors):
    min_dist = 10000 # to act like positive infinity
    min_index = -1
    query_vector = vectors[word_index]
    for index, vector in enumerate(vectors):
        if euclidean_dist(vector, query_vector) < min_dist and not np.array_equal(vector, query_vector):
            min_dist = euclidean_dist(vector, query_vector)
            min_index = index
    return min_index
  
print(int2word[find_closest(word2int['cat'], vectors)])


5
[['the', 'cat', 'sat', 'on', 'the', 'mat']]
[['the', 'cat'], ['the', 'sat'], ['cat', 'the'], ['cat', 'sat'], ['cat', 'on'], ['sat', 'the'], ['sat', 'cat'], ['sat', 'on'], ['sat', 'the'], ['on', 'cat'], ['on', 'sat'], ['on', 'the'], ['on', 'mat'], ['the', 'sat'], ['the', 'on'], ['the', 'mat'], ['mat', 'on'], ['mat', 'the']]
(18, 5) (18, 5)
[2.2158246 1.491334  0.9773514 2.0339558 1.3369646]
mat
